In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy
from tqdm import tqdm
import random
from random import shuffle

In [2]:
import cv2
import numpy as np
import random
import os
import re
from tqdm import tqdm
from copy import deepcopy
import sys

import cv2
import numpy as np
from tqdm import tqdm
from multiprocessing import Process, Manager


In [3]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

def live_plot(data, figsize=(16,8), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    plt.imshow(convert_to_rgb(data.astype(int)))
    plt.title(title)
    plt.show()

In [4]:
manager = Manager()  # create only 1 mgr
# prob_dict = manager.dict()

## 读取数据

In [5]:
def convert_to_rgb(img):
    img = deepcopy(img)
    b,g,r = cv2.split(img)  
    img = cv2.merge([r,g,b])
    return img

In [6]:
background = cv2.imread('../../base_data/background.jpeg')
hedge = cv2.imread('../../base_data/hedge.jpeg')

## 生成视频代码

In [7]:
def generate_video(frame_dir=None, video_dir=None, video_name='video.avi'):

    image_num = len([i for i in os.listdir(frame_dir) if '.jpg' in i])
    image_name_list = [f'{i}.jpg' for i in range(image_num)]
    frame = cv2.imread(os.path.join(frame_dir, image_name_list[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(f'{video_dir}/{video_name}', 0, 20, (width, height))

    # video = cv2.VideoWriter(f'{self.video_dir}/{video_name}', 0, 20, (width, height))

    for image in image_name_list:
        video.write(cv2.imread(os.path.join(frame_dir, image)))

    cv2.destroyAllWindows()
    video.release()

## 定义参数

In [8]:
speed_dict = {
    "background": 1,
    "hedge_1": 2,
    "hedge_2": 3,
    "black_bar": 4
}

crop_height = 900
crop_width = 1500
slide_width = 800
camera_width = 250

crop_width_extend = crop_width + 1000

size_1 = camera_width * 2
size_2 = slide_width

dimension = 3
black_bar_width = 10

background_prob = 0.7
noise_average_size = 10

vertical_movement = [0]
horizontal_movement = [1, 2]

In [9]:
background_crop = deepcopy(background[-crop_height:, :crop_width, :])
# live_plot(background_crop)

In [10]:
hedge_crop = deepcopy(hedge[-crop_height:, :crop_width, :])
# live_plot(hedge_crop)

In [11]:
def rgb_mean(image):
    return tuple([np.mean([np.mean(image[:, :, i])]) for i in range(2, -1, -1)])

def initialise_image(image):
    reshape_size = image.shape[0] * image.shape[1]
    a = np.zeros((dimension, reshape_size))
    for i in range(3):
        a[i] = np.array(image[:, :, i]).reshape(1, reshape_size)
    return a

def random_select():
    return random.randint(0, 250), random.randint(0, 750)

def random_radius():
    return random.randint(10, 35)

def mahalanobis(x, mean, cov):
    return np.sqrt(np.linalg.inv(cov).dot(np.transpose(x - mean)).dot((x - mean)))

def cal_ratio_prob(x):
    return 1 - (x/x.sum())

def ratio_soft_max(self, x):
    return ratio_prob(soft_max(x))

def soft_max(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [12]:
hedge_mean = np.array(rgb_mean(hedge_crop))
background_mean = np.array(rgb_mean(background_crop))
hedge_cov = np.cov(initialise_image(hedge_crop))
background_cov = np.cov(initialise_image(background_crop))

## 显示图片

In [13]:
# plt.imshow(convert_to_rgb(background))
# plt.show()
# plt.imshow(convert_to_rgb(hedge))
# plt.show()

In [14]:
background.shape, hedge.shape

((941, 1672, 3), (1128, 1504, 3))

## 定义参数

In [15]:
speed_dict = {
    "background": 1,
    "hedge_1": 2,
    "hedge_2": 3,
    "black_bar": 4
}

crop_height = 900
crop_width = 1500
slide_width = 800
camera_width = 250

crop_width_extend = crop_width + 1000

size_1 = camera_width * 2
size_2 = slide_width

dimension = 3
black_bar_width = 10

background_prob = 0.7
noise_average_size = 10

vertical_movement = [0]
horizontal_movement = [1, 2]

In [16]:
background_crop = deepcopy(background[-crop_height:, :crop_width, :])
# live_plot(background_crop)

In [17]:
hedge_crop = deepcopy(hedge[-crop_height:, :crop_width, :])
# live_plot(hedge_crop)

## 检测位移的算法

In [18]:
def calculate_frame_prob(frame):
    frame_prob = np.zeros(frame.shape)
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            pixel = frame[i][j]
            hedge_distance = mahalanobis(pixel, hedge_mean, hedge_cov)
            top_distance = mahalanobis(pixel, background_mean, background_cov)
            ratio_prob = cal_ratio_prob(np.array([top_distance, hedge_distance]))[0]
            frame_prob[i][j] = round(ratio_prob)
    return frame_prob

In [ ]:
m = 10
accuracy_dict = {}
for sn in tqdm(range(100,1,-1), ncols=70):
    frame_dir = '../scene_data/{}'.format(sn)
    
    scene_dict = {}
    real_back_prob_list = []
    for file_name in os.listdir(frame_dir):
        if file_name[0] != '.':
            num, verti_pos, hori_pos, real_back_prob = file_name.replace('.jpg', '').split('_')
            real_back_prob_list.append(int(real_back_prob))
            scene_dict[int(num)] = {'file_name': file_name, 'num':num, 'verti_pos':int(verti_pos), 'hori_pos':int(hori_pos)}
    real_back_prob_mean = int(np.mean(real_back_prob_list))
    accuracy_dict[real_back_prob_mean] = {}
    
    for n in tqdm([8,4,2,1], ncols=70):
        set_shape = (2*n, 2*n, 3)
        result_compare_list = []

        for _ in tqdm(range(100), ncols=70):
            compare_list = []
            scene_num = random.choice(range(min(scene_dict.keys()), max(scene_dict.keys())-1))
            for num in [scene_num, scene_num + 1]:
                file_name = scene_dict[num]['file_name']
                verti_pos = scene_dict[num]['verti_pos']
                hori_pos = scene_dict[num]['hori_pos']
                frame = np.array(cv2.imread(f"{frame_dir}/{file_name}"))
                compare_list.append([frame, (verti_pos, hori_pos), num])
            ground_truth = np.array(compare_list[1][1]) - np.array(compare_list[0][1])

            compare_dict = {}
            frame = compare_list[0][0]
            compare_frame = compare_list[1][0]
            for _ in range(32):
                break_flag = 1
                for _ in range(128):
                    frame_i, frame_j = random.choice(range(frame.shape[0])), random.choice(range(frame.shape[1]))
                    pixels = frame[frame_i - n:frame_i + n, frame_j - n:frame_j + n, :]
                    pixels_prob = calculate_frame_prob(pixels)
                    _ratio = min(1, float(real_back_prob_mean/100)*1.1)
                    if pixels.shape == set_shape and np.sum(pixels_prob) >= set_shape[0] * set_shape[1] * set_shape[2] * _ratio:
                        break_flag = 0
                        break
                if break_flag:
                    compare_dict = {}
                    print('break')
                    break
                    
                for i in range(frame_i-m, frame_i+m):
                    for j in range(frame_j-m, frame_j+m):
                        compare_pixels = compare_frame[i - n:i + n,j - n:j + n,:]
                        if pixels.shape == set_shape and compare_pixels.shape == set_shape:
                            diff_array = np.array([i for i in ((pixels - compare_pixels)).reshape(-1,1) if i != 0])
                            diff = np.sum(np.power(diff_array,2))/len(diff_array)
                            compare_dict[(frame_i - i, frame_j - j)] = compare_dict.get((frame_i - i, frame_j - j), []) + [diff]
            compare_dict = {k:np.mean(v) for k,v in compare_dict.items()}
            compare_dict = {k: v for k, v in sorted(compare_dict.items(), key=lambda item: item[1])}
            if list(compare_dict.keys()):
                predict = np.array(list(compare_dict.keys())[0])
#                 print((predict, ground_truth))
                result_compare_list.append(list(ground_truth) == list(predict))
        accuracy = np.nan
        if len(result_compare_list):
            accuracy = sum(result_compare_list)*100/len(result_compare_list)
        accuracy_dict[real_back_prob_mean][n] = accuracy
        df = pd.DataFrame().from_dict(accuracy_dict)
        df.to_csv('accuracy_1.csv', index=True)

  0%|                                           | 0/4 [00:00<?, ?it/s]

  0%|                                         | 0/100 [00:00<?, ?it/s]

  1%|▎                              | 1/100 [01:13<2:00:41, 73.15s/it]

  2%|▌                              | 2/100 [02:28<2:00:46, 73.94s/it]

  3%|▉                              | 3/100 [03:38<1:57:11, 72.49s/it]

  4%|█▏                             | 4/100 [04:55<1:58:34, 74.11s/it]

  5%|█▌                             | 5/100 [06:06<1:55:26, 72.91s/it]

  6%|█▊                             | 6/100 [07:22<1:56:07, 74.12s/it]

  7%|██▏                            | 7/100 [08:34<1:53:29, 73.22s/it]

  8%|██▍                            | 8/100 [09:56<1:56:16, 75.84s/it]

  9%|██▊                            | 9/100 [11:12<1:55:30, 76.16s/it]

 10%|███                           | 10/100 [12:28<1:53:47, 75.86s/it]

 11%|███▎                          | 11/100 [13:46<1:53:25, 76.47s/it]

 12%|███▌                          | 12/100 [15:06<1:53:43, 77.5

  5%|█▋                               | 5/100 [00:27<08:56,  5.65s/it]

  6%|█▉                               | 6/100 [00:33<08:58,  5.73s/it]

  7%|██▎                              | 7/100 [00:38<08:35,  5.55s/it]

  8%|██▋                              | 8/100 [00:44<08:40,  5.66s/it]

  9%|██▉                              | 9/100 [00:50<08:20,  5.50s/it]

 10%|███▏                            | 10/100 [00:55<08:25,  5.61s/it]

 11%|███▌                            | 11/100 [01:01<08:07,  5.48s/it]

 12%|███▊                            | 12/100 [01:06<08:12,  5.59s/it]

 13%|████▏                           | 13/100 [01:12<07:54,  5.46s/it]

 14%|████▍                           | 14/100 [01:17<08:00,  5.59s/it]

 15%|████▊                           | 15/100 [01:23<08:03,  5.69s/it]

 16%|█████                           | 16/100 [01:29<07:44,  5.53s/it]

 17%|█████▍                          | 17/100 [01:34<07:48,  5.64s/it]

 18%|█████▊                          | 18/100 [01:40<07:49,  5.7

  8%|██▍                            | 8/100 [09:23<1:47:06, 69.86s/it]

  9%|██▊                            | 9/100 [10:33<1:46:03, 69.93s/it]

 10%|███                           | 10/100 [11:43<1:44:52, 69.91s/it]

 11%|███▎                          | 11/100 [12:53<1:43:45, 69.95s/it]

 12%|███▌                          | 12/100 [14:03<1:42:37, 69.97s/it]

 13%|███▉                          | 13/100 [15:14<1:41:51, 70.24s/it]

 14%|████▏                         | 14/100 [16:24<1:40:39, 70.22s/it]

 15%|████▌                         | 15/100 [17:35<1:39:44, 70.40s/it]

 16%|████▊                         | 16/100 [18:44<1:37:58, 69.99s/it]

 17%|█████                         | 17/100 [19:53<1:36:28, 69.74s/it]

 18%|█████▍                        | 18/100 [21:04<1:35:45, 70.07s/it]

 19%|█████▋                        | 19/100 [22:14<1:34:35, 70.07s/it]

 20%|██████                        | 20/100 [23:24<1:33:23, 70.04s/it]

 21%|██████▎                       | 21/100 [24:34<1:32:11, 70.0

 14%|████▍                           | 14/100 [01:15<07:56,  5.54s/it]

 15%|████▊                           | 15/100 [01:21<07:39,  5.40s/it]

 16%|█████                           | 16/100 [01:26<07:26,  5.31s/it]

 17%|█████▍                          | 17/100 [01:32<07:33,  5.47s/it]

 18%|█████▊                          | 18/100 [01:37<07:18,  5.35s/it]

 19%|██████                          | 19/100 [01:42<07:07,  5.27s/it]

 20%|██████▍                         | 20/100 [01:48<07:15,  5.45s/it]

 21%|██████▋                         | 21/100 [01:53<07:01,  5.34s/it]

 22%|███████                         | 22/100 [01:58<07:07,  5.48s/it]

 23%|███████▎                        | 23/100 [02:04<06:55,  5.40s/it]

 24%|███████▋                        | 24/100 [02:10<07:01,  5.55s/it]

 25%|████████                        | 25/100 [02:15<06:46,  5.42s/it]

 26%|████████▎                       | 26/100 [02:21<06:50,  5.55s/it]

 27%|████████▋                       | 27/100 [02:26<06:34,  5.4

 19%|█████▋                        | 19/100 [24:12<1:50:35, 81.92s/it]

 20%|██████                        | 20/100 [25:34<1:48:56, 81.70s/it]

 21%|██████▎                       | 21/100 [26:56<1:47:41, 81.79s/it]

 22%|██████▌                       | 22/100 [28:19<1:46:47, 82.14s/it]

 23%|██████▉                       | 23/100 [29:40<1:45:19, 82.07s/it]

 24%|███████▏                      | 24/100 [31:04<1:44:20, 82.37s/it]

 25%|███████▌                      | 25/100 [32:27<1:43:30, 82.81s/it]

 26%|███████▊                      | 26/100 [33:49<1:41:32, 82.33s/it]

 27%|████████                      | 27/100 [35:12<1:40:42, 82.77s/it]

 28%|████████▍                     | 28/100 [36:36<1:39:27, 82.89s/it]

 29%|████████▋                     | 29/100 [37:58<1:38:05, 82.90s/it]

 30%|█████████                     | 30/100 [39:23<1:37:08, 83.26s/it]

 31%|█████████▎                    | 31/100 [40:44<1:35:17, 82.86s/it]

 32%|█████████▌                    | 32/100 [42:08<1:34:15, 83.1

 25%|████████                        | 25/100 [02:36<07:56,  6.35s/it]

 26%|████████▎                       | 26/100 [02:42<08:01,  6.50s/it]

 27%|████████▋                       | 27/100 [02:49<07:45,  6.37s/it]

 28%|████████▉                       | 28/100 [02:55<07:33,  6.29s/it]

 29%|█████████▎                      | 29/100 [03:02<07:39,  6.47s/it]

 30%|█████████▌                      | 30/100 [03:08<07:23,  6.34s/it]

 31%|█████████▉                      | 31/100 [03:14<07:11,  6.25s/it]

 32%|██████████▏                     | 32/100 [03:20<07:01,  6.20s/it]

 33%|██████████▌                     | 33/100 [03:27<07:08,  6.39s/it]

 34%|██████████▉                     | 34/100 [03:33<06:57,  6.32s/it]

 35%|███████████▏                    | 35/100 [03:39<06:59,  6.46s/it]

 36%|███████████▌                    | 36/100 [03:45<06:44,  6.33s/it]

 37%|███████████▊                    | 37/100 [03:52<06:33,  6.25s/it]

 38%|████████████▏                   | 38/100 [03:58<06:24,  6.2

 30%|█████████                     | 30/100 [39:39<1:31:23, 78.33s/it]

 31%|█████████▎                    | 31/100 [40:49<1:27:11, 75.82s/it]

 32%|█████████▌                    | 32/100 [42:02<1:25:00, 75.00s/it]

 33%|█████████▉                    | 33/100 [43:14<1:22:26, 73.82s/it]

 34%|██████████▏                   | 34/100 [44:27<1:20:56, 73.58s/it]

 35%|██████████▌                   | 35/100 [45:40<1:19:48, 73.68s/it]

 36%|██████████▊                   | 36/100 [46:53<1:18:22, 73.47s/it]

 37%|███████████                   | 37/100 [48:06<1:16:43, 73.07s/it]

 38%|███████████▍                  | 38/100 [49:20<1:15:46, 73.33s/it]

 39%|███████████▋                  | 39/100 [50:32<1:14:24, 73.18s/it]

 40%|████████████                  | 40/100 [51:44<1:12:49, 72.83s/it]

 41%|████████████▎                 | 41/100 [53:00<1:12:32, 73.77s/it]

 42%|████████████▌                 | 42/100 [54:27<1:14:55, 77.51s/it]

 43%|████████████▉                 | 43/100 [55:53<1:16:03, 80.0

 36%|███████████▌                    | 36/100 [03:18<05:50,  5.48s/it]

 37%|███████████▊                    | 37/100 [03:23<05:38,  5.38s/it]

 38%|████████████▏                   | 38/100 [03:29<05:42,  5.52s/it]

 39%|████████████▍                   | 39/100 [03:35<05:45,  5.66s/it]

 40%|████████████▊                   | 40/100 [03:41<05:46,  5.78s/it]

 41%|█████████████                   | 41/100 [03:46<05:29,  5.58s/it]

 42%|█████████████▍                  | 42/100 [03:52<05:27,  5.65s/it]

 43%|█████████████▊                  | 43/100 [03:57<05:13,  5.49s/it]

 44%|██████████████                  | 44/100 [04:03<05:13,  5.60s/it]

 45%|██████████████▍                 | 45/100 [04:08<05:00,  5.46s/it]

 46%|██████████████▋                 | 46/100 [04:14<05:01,  5.58s/it]

 47%|███████████████                 | 47/100 [04:19<04:49,  5.45s/it]

 48%|███████████████▎                | 48/100 [04:25<04:49,  5.57s/it]

 49%|███████████████▋                | 49/100 [04:30<04:37,  5.4